In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import sklearn
import category_encoders
import optuna

In [ ]:
df = pd.read_csv('/kaggle/input/horse-colic/horse.csv')

In [ ]:
df.head()

In [ ]:
df['outcome'].value_counts()

In [ ]:
def nulls(df):
    null = df.isnull().sum().sort_values()[::-1]
    null = null[null > 0] / len(df)
    null = pd.concat([df[null.index].dtypes, null],axis=1)
    null.columns=['dtype', 'ratio_null']
    null.index.name = 'feature'
    if null.empty:
        return "No nulls."
    return null

In [ ]:
nulls(df)

In [ ]:
null_object = null[null['dtype'] == 'object'].index.to_list()
null_float = null[null['dtype'] == 'float64'].index.to_list()

In [ ]:
fill_floats = df[null_float].apply(lambda x: x.median())

In [ ]:
df[null_float] = df[null_float].fillna(fill_floats); df.head()

In [ ]:
fill_objects = df[null_object].apply(lambda x: x.mode()); fill_objects
fill_objects = fill_objects.transpose()[0]  # convert to series
fill_objects

In [ ]:
df[null_object] = df[null_object].fillna(fill_objects); df.head()

In [ ]:
nulls(df)

In [ ]:
df.get

In [ ]:
y = df['outcome']
X = df.drop('outcome', axis=1)

In [ ]:
nuniques = X.apply(lambda x: x.nunique())
nuniques_binary = nuniques[nuniques == 2]
binary_features = nuniques_binary.index.to_list()
binary_features

In [ ]:
X[binary_features] = OrdinalEncoder().fit_transform(X[binary_features])

In [ ]:
X = OneHotEncoder().fit_transform(X)  # 67 features, 299 rows
print(X.shape)
y = LabelEncoder().fit_transform(y)  # classes into 0,1,2

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=123)
print(f"number of training: \t{len(y_train)}")
print(f"number of validation: \t{len(y_valid)}")

In [ ]:
def objective(trial):
    rf_max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
    n_estimators = trial.suggest_int('n_estimators', 2, 100, log=True)
    specified_max_depth = trial.suggest_categorical('max_depth_bool', [False, True])
    if specified_max_depth:
        rf_max_depth = rf_max_depth
    else:
        rf_max_depth=None
    classifier_obj = sklearn.ensemble.RandomForestClassifier(max_depth=rf_max_depth, n_estimators=n_estimators)
    score = sklearn.model_selection.cross_val_score(classifier_obj, X_train, y_train, scoring='f1_weighted', n_jobs=-1, cv=3)
    return score.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
best_params = study.best_trial.params
if not best_params['max_depth_bool']:
    best_params['max_depth'] = None
del best_params['max_depth_bool']
best_params

In [ ]:
model = RandomForestClassifier(**best_params)
model.fit(X_train, y_train)
model.score(X_valid, y_valid)